In [1]:
import pandas as pd
import numpy as np

from prep import load_model
import plotly.express as px
import plotly.graph_objects as pgo

ModuleNotFoundError: No module named 'prep'

In [2]:
# load trained model for explaination
model, data = load_model()

# get trined embedding of side effects
side_effect_embedding = pd.DataFrame(model.mip.weight.data.numpy())

# side effect mapping
se_name2idx, se_idx2name = {}, {}
for se, idx in data.side_effect_name_to_idx.items():
    se = se.lower()
    se_name2idx[se] = idx
    se_idx2name[idx] = se


In [3]:
import pickle
side_effect = [data.side_effect_idx_to_name[i].lower() for i in range(860)]

# get side effect category
with open('../github/nyxflower/data/disease/ontology/disease2ONidx.pkl', 'rb') as f:
    disease_to_onidx = pickle.load(f)
with open('../github/nyxflower/data/disease/ontology/ONidx2disease.pkl', 'rb') as f:
    onidx_to_disease = pickle.load(f)   


In [4]:
# get side effect catagory with adjusted side effect name in disease cata
side_effect_fixed = pd.read_csv('./se_tmp.tsv', sep='\t')['se']

onto = []
if_onto = []
for se in side_effect_fixed:
    onto.append(disease_to_onidx.get(se.lower()))
    

In [7]:
# add side effect in psychiatry and psychology catagory
psychiaty_psychology = "panic disorder,Anxiety,Bipolar Disorder,major depression,alcohol consumption,schizoaffective disorder,borderline personality disorder,Agoraphobia,conversion disorder".lower().split(',')

asssert: len(onto) == len(side_effect)

for i in psychiaty_psychology:
    i = se_name2idx[i]
    onto[i] = 'F'



In [5]:
se = []
cata = []
for s, ons in zip(side_effect, onto):
    if ons:
        for on in ons:
            se.append(s)
            cata.append(on[:3])

    


In [8]:
data_se_ca = pd.DataFrame({'side effect': se, 'cata': cata})
tmp = data_se_ca['side effect'].unique().shape[0] - len(psychiaty_psychology)
print(f'There are {len(psychiaty_psychology)} mental disorders, {tmp} diseases, and {861-tmp-2*len(psychiaty_psychology)} other observed symptoms.' )


There are 9 mental disorders, 518 diseases, and 325 other observed symptoms.


In [ ]:
tmp = data_se_ca['cata'].value_counts()
hist_se_cata = pd.DataFrame({'count': tmp.values}, index=tmp.index)

cata_name = [onidx_to_disease.get(idx) for idx in hist_se_cata.index]
cata_name[-3] = 'mental discorders'

hist_se_cata['side effect category (disease and mental disorders)'] = cata_name
hist_se_cata.sort_index(inplace=True)

fig = px.bar(hist_se_cata, x='side effect category (disease and mental disorders)', y='count', text='count')
fig.update_xaxes(tickangle=45)

fig.show()

In [8]:
# Table for all

onidx_to_disease['F'] = 'mental discorders'
cata_name = [onidx_to_disease[i] for i in cata]

df = pd.DataFrame({'side effect': se, 'category': cata, 'category name': cata_name})
df = df.drop_duplicates()
df['side effect id'] = [se_name2idx[i] for i in df['side effect']]
# df.to_csv('side_effect+catagory.tsv', sep='\t', index=False)

eva_layer_df = pd.read_csv('data/with.csv')
eva_layer_df['PIU score'] = (eva_layer_df['auroc_layer-2'] - eva_layer_df['auroc_layer-0']) / eva_layer_df['auroc_layer-2']
eva_layer_df = eva_layer_df[['side_effect', 'n_instance', 'auroc_layer-2', 'PIU score']]
eva_layer_df.columns = ['side effect', '#instance', 'AUROC', 'PIU score']
eva_layer_df['side effect'] = eva_layer_df['side effect'].str.lower()

data_all = df.merge(eva_layer_df, on='side effect')
data_all


NameError: name 'onidx_to_disease' is not defined

## Side Effect Embedding

In [11]:
import scipy.sparse as sp
from umap import UMAP

### From side effect - GO association

In [11]:
# load all side effect -- GO association
se_go_all = sp.load_npz("./out/coo.npz")
se_go_all.shape


(861, 2001302)

In [12]:
se_go_all = pd.DataFrame(se_go_all.toarray()).astype(int)
se_go_all = se_go_all.loc[(se_go_all!=0).any(1)]
se_go_all = se_go_all.loc[:, (se_go_all!=0).any(0)]
se_go_all.shape

(44, 26)

In [15]:
se_go_all

,4089,4601,4659,5251,5328,5789,5887,6805,8504,15020,...,32264,33280,47555,55085,55114,70062,98591,99601,140078,2001225
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
76,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
102,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
103,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
121,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
186,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
umap_3d_go = UMAP(n_components=3, init='random', random_state=8)
se_3d_umap_go = umap_3d_go.fit_transform(se_go_all)

In [25]:
coo_go = se_3d_umap_go

In [26]:

fig = px.scatter_3d(x=coo_go[:, 0], y=coo_go[:, 1], z=coo_go[:, 2], hover_name=[se_idx2name[i] for i in se_go_all.index])
fig.show()

### From trained side effect embeddings

In [115]:
side_effect_embedding.shape

(861, 160)

In [138]:
# get lower dimentional embeddings from  
umap_3d = UMAP(n_components=3, init='random', random_state=8)
se_3d_umap = umap_3d.fit_transform(side_effect_embedding)

In [139]:
coo = se_3d_umap[data_all['side effect id']]
coo

array([[ -0.45648676,  -9.674313  ,   6.0638475 ],
       [  1.0522346 , -10.952207  ,   8.1596    ],
       [ -2.1450408 , -10.336667  ,   5.8824964 ],
       ...,
       [  0.17618823,  -9.858621  ,   8.860392  ],
       [  0.56307656,  -9.646261  ,   8.165037  ],
       [  1.2837819 , -10.469733  ,   7.619313  ]], dtype=float32)

In [140]:
fig = px.scatter_3d(x=coo[:, 0], y=coo[:, 1], z=coo[:, 2], color=data_all['category'], hover_name=data_all['side effect'])
fig.show()


## Case Study I: Infection Diseases

In [12]:
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AffinityPropagation

In [13]:
c1_idx = df[df['category'] == 'C01']['side effect id']
ses = ','.join(c1_idx.astype(str).tolist())
c1_idx.values, c1_idx.shape

(array([ 18,  27,  34,  60,  93,  98, 101, 115, 120, 128, 133, 154, 170,
        176, 182, 184, 223, 228, 268, 270, 279, 337, 347, 369, 370, 386,
        388, 429, 496, 543, 544, 556, 557, 645, 650, 660, 685, 710, 712,
        713, 714, 728, 737, 793, 801, 815, 828, 845, 849, 850]),
 (50,))

In [27]:
ses

'18,27,34,60,93,98,101,115,120,128,133,154,170,176,182,184,223,228,268,270,279,337,347,369,370,386,388,429,496,543,544,556,557,645,650,660,685,710,712,713,714,728,737,793,801,815,828,845,849,850'

#### Biological Process GO items for Training

In [20]:
se, go, da = np.array([], dtype=int), np.array([], dtype=int), np.array([], dtype=float)
go_dict = {}
go_full_dict = {}
go_name_dict = {}

for se_idx in c1_idx.values:
    file = './c1_out/all-all-{}-2.0-0.99/name_space.csv'.format(se_idx)
    if os.path.exists(file):
        go_idx = pd.read_csv(file, usecols=[1, 2, 3])
        # print(go_idx.head())
        go_name = go_idx['name'][go_idx['namespace'] == 'biological_process'].to_numpy().flatten()
        go_idx = go_idx['id'][go_idx['namespace'] == 'biological_process'].to_numpy().flatten()
        # print(go_idx)
        if len(go_idx):
            for i, j in zip(go_idx, go_name):
                tmp_int = int(i[3:])
                if go_full_dict.get(tmp_int) is None:
                    go_full_dict[tmp_int] = i
                if go_name_dict.get(i) is None:
                    go_name_dict[i] = j

            go_idx = [int(i[3:]) for i in go_idx]
            c = Counter(go_idx)
            go_idx, go_count = list(zip(*c.items()))

            go_new_id = []
            for i in go_idx:
                if go_dict.get(i) is None:
                    go_dict[i] = len(go_dict)
                go_new_id.append(go_dict[i])

            se = np.concatenate((se, [se_idx] * len(go_new_id)))
            go = np.concatenate((go, go_new_id))
            da = np.concatenate((da, go_count))

coo = sp.coo_matrix((da, (se, go)), shape=(data.n_et, len(go_dict)))
# sp.save_npz('./out/coo', coo)
# print(coo.toarray())
inv_go_dict = {v:k for k,v in go_dict.items()}

In [21]:
c1_go = pd.DataFrame(coo.toarray()).astype(int)
c1_go = c1_go.loc[(c1_go!=0).any(1)]
c1_go.shape

(46, 221)

In [22]:
seed = 20

# === standardization representation matrix ===
# scaler = StandardScaler()
# c1_go_scale = scaler.fit_transform(c1_go.values)

# === clustering ===
clustering = AffinityPropagation(random_state=seed).fit(c1_go.values)

# === dimensional reduction ===
umap_3d = UMAP(n_components=2, init='random', random_state=seed)
coo = umap_3d.fit_transform(c1_go.values)
# coo_scale = umap_3d.fit_transform(c1_go_scale)

In [23]:
c1_cluster = pd.DataFrame({'side effect idx': c1_go.index, 'side effect': [se_idx2name[i] for i in c1_go.index], 'cluster': [str(i) for i in clustering.labels_]})
c1_go.columns = [go_full_dict[inv_go_dict[i]] for i in range(len(c1_go.columns))]
c1_go.index = range(c1_go.shape[0])
c1_cluster = pd.concat((c1_cluster, c1_go), axis=1)
c1_cluster


,side effect idx,side effect,cluster,GO:0055085,GO:0015721,GO:0015711,GO:0052695,GO:0043252,GO:0008202,GO:0140115,...,GO:0071397,GO:0035176,GO:0046684,GO:0015842,GO:0045472,GO:0035690,GO:0042755,GO:0045541,GO:0042713,GO:0046219
0,18,peritonitis,8,9,5,3,3,3,3,2,...,0,0,0,0,0,0,0,0,0,0
1,27,sepsis,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34,appendicitis,1,12,5,0,4,0,6,0,...,0,0,0,0,0,0,0,0,0,0
3,60,neumonia,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,93,septic shock,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,98,pneumonia staphylococcal,8,8,4,3,3,3,4,2,...,0,0,0,0,0,0,0,0,0,0
6,101,lobar pneumonia,8,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,115,pleurisy,0,10,5,0,3,0,8,0,...,0,0,0,0,0,0,0,0,0,0
8,120,aspiration pneumonia,8,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,128,candida infection,1,0,0,0,4,0,6,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# === plot figure ===
fig = px.scatter(x=coo[:, 0], y=coo[:, 1], text=c1_cluster['side effect'], color=c1_cluster['cluster'])
fig.update_traces(textposition='bottom center')
fig.update_layout(autosize=False, width=900, height=1000)
fig.show()
# fig = px.scatter(x=coo_scale[:, 0], y=coo_scale[:, 1], text=[se_idx2name[i] for i in c1_go.index])
# fig.update_traces(textposition='top center')
# fig.show()


In [25]:
c1_go_com = pd.DataFrame()
for i in range(10):
    # c1_cluster[c1_cluster['cluster'] == '3'][c1_go.columns].sum().sort_values(ascending=False)
    c1_go_com[i] = c1_cluster[c1_cluster['cluster'] == str(i)][c1_go.columns].sum()
c1_go_com[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]


,0,1,2,3,4,5,6,7,8,9
GO:0055085,10,55,0,14,13,10,0,0,58,10
GO:0015721,5,25,0,6,7,5,0,0,27,0
GO:0015711,0,8,0,0,5,4,0,0,12,3
GO:0052695,3,49,3,4,4,4,5,3,37,3
GO:0043252,0,7,0,4,4,0,0,0,6,3
...,...,...,...,...,...,...,...,...,...,...
GO:0035690,0,0,0,0,0,0,0,0,0,4
GO:0042755,0,0,0,0,0,0,0,0,0,3
GO:0045541,0,0,0,0,0,0,0,0,0,2
GO:0042713,0,0,0,0,0,0,0,0,0,2


In [26]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

c1_cat = pd.concat(((c1_go_com > 0).sum(axis=1), c1_go_com.sum(axis=1)), axis=1)
c1_cat.columns = ['cluster count', 'gene count']
c1_cat['GO name'] = [go_name_dict[i] for i in c1_cat.index]
c1_cat.sort_values(by=['cluster count', 'gene count'], ascending=False)

,cluster count,gene count,GO name
GO:0052695,10,115,cellular glucuronidation
GO:0055114,9,361,oxidation-reduction process
GO:0006805,9,248,xenobiotic metabolic process
GO:0042738,9,147,exogenous drug catabolic process
GO:1902476,9,98,chloride transmembrane transport
GO:0042391,9,73,regulation of membrane potential
GO:0050877,9,69,nervous system process
GO:0007214,9,57,gamma-aminobutyric acid signaling pathway
GO:0008202,8,152,steroid metabolic process
GO:0034220,8,113,ion transmembrane transport


In [42]:
fig = pgo.Figure()
for i in [0, 5, 1, 7, 3]:
    count = c1_cluster[c1_cluster['cluster'] == str(i)][c1_go.columns].sum()
    fig.add_traces(pgo.Bar(name=f'Cluster-{i}', x=c1_go.columns, y = count.values))
fig.update_layout(barmode='group', autosize=False, width=6000, height=700)
fig.show()

#### Cellular Component

In [79]:
c1_idx.values

array([ 18,  27,  34,  60,  93,  98, 101, 115, 120, 128, 133, 154, 170,
       176, 182, 184, 223, 228, 268, 270, 279, 337, 347, 369, 370, 386,
       388, 429, 496, 543, 544, 556, 557, 645, 650, 660, 685, 710, 712,
       713, 714, 728, 737, 793, 801, 815, 828, 845, 849, 850])

In [80]:
cse, cgo, cda = np.array([], dtype=int), np.array([], dtype=int), np.array([], dtype=float)
cgo_dict = {}
cgo_full_dict = {}
cgo_name_dict = {}

for cse_idx in c1_idx.values:
    file = './out_new/all-all-{}-2.0-0.99/name_space.csv'.format(cse_idx)
    if os.path.exists(file):
        cgo_idx = pd.read_csv(file, usecols=[1, 2, 3])
        # print(go_idx.head())
        cgo_name = cgo_idx['name'][cgo_idx['namespace'] == 'cellular_component'].to_numpy().flatten()
        cgo_idx = cgo_idx['id'][cgo_idx['namespace'] == 'cellular_component'].to_numpy().flatten()
        if len(cgo_idx):
            for i, j in zip(cgo_idx, cgo_name):
                tmp_int = int(i[3:])
                if cgo_full_dict.get(tmp_int) is None:
                    cgo_full_dict[tmp_int] = i
                if cgo_name_dict.get(i) is None:
                    cgo_name_dict[i] = j

            cgo_idx = [int(i[3:]) for i in cgo_idx]
            c = Counter(cgo_idx)
            # print(c)
            cgo_idx, cgo_count = list(zip(*c.items()))
            # print(cgo_count)

            cgo_new_id = []
            for i in cgo_idx:
                if cgo_dict.get(i) is None:
                    cgo_dict[i] = len(cgo_dict)
                cgo_new_id.append(cgo_dict[i])

            cse = np.concatenate((cse, [cse_idx] * len(cgo_new_id)))
            cgo = np.concatenate((cgo, cgo_new_id))
            cda = np.concatenate((cda, cgo_count))

ccoo = sp.coo_matrix((cda, (cse, cgo)), shape=(data.n_et, len(cgo_dict)))
# sp.save_npz('./out/coo', coo)
# print(coo.toarray())
inv_cgo_dict = {v:k for k,v in cgo_dict.items()}

In [81]:
c1_cgo = pd.DataFrame(ccoo.toarray()).astype(int)
c1_cgo = c1_cgo.loc[(c1_cgo!=0).any(1)]

c1_ccluster = pd.DataFrame({'side effect idx': c1_cgo.index, 'side effect': [se_idx2name[i] for i in c1_cgo.index]})
c1_cgo.columns = [cgo_full_dict[inv_cgo_dict[i]] for i in range(len(c1_cgo.columns))]
c1_cgo.index = range(c1_cgo.shape[0])
c1_ccluster = pd.concat((c1_ccluster, c1_cgo), axis=1)
c1_ccluster

,side effect idx,side effect,GO:0016323,GO:0098591,GO:0009986,GO:0016324,GO:0070062,GO:0005789,GO:0005891,GO:0005739,GO:0005887,GO:0034707,GO:0005886,GO:0098982,GO:0043005,GO:0032590,GO:1902711,GO:0045202,GO:0099056,GO:0099154,GO:0097444,GO:0008076,GO:0034704,GO:0043231,GO:1990454,GO:0098794,GO:0005783,GO:0016529,GO:0005623,GO:0099060,GO:0032809,GO:0031526,GO:0002095,GO:0005833,GO:0072562,GO:0001518,GO:0098983,GO:0005901,GO:0046581,GO:0065010
0,18,peritonitis,5,2,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27,sepsis,0,0,9,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,34,appendicitis,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,60,neumonia,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,93,septic shock,0,0,0,0,0,0,5,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,98,pneumonia staphylococcal,4,2,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,101,lobar pneumonia,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,115,pleurisy,8,2,0,0,0,0,0,0,36,9,59,7,12,5,4,12,5,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,120,aspiration pneumonia,0,0,0,0,0,11,6,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,128,candida infection,0,0,0,0,0,19,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [97]:
se_5789 = c1_ccluster[c1_ccluster['GO:0005789'] > 0]['side effect idx'].values

array([ 34, 101, 120, 128, 154, 170, 184, 279, 337, 370, 429, 544, 645,
       650, 685, 710, 712, 713, 728, 801, 815, 845, 849, 850])

In [ ]:
# Drugs which associated with 'GO:005789'



In [61]:
for i, j in list(zip(c1_cluster['side effect idx'].values.tolist(), c1_idx.values.tolist())):
    if i != j:
        print(f"{se_idx2name[j]} -- No.{j}, the only side effect which is not associated any biological process GO item.\n")
        break

print(se_idx2name[223], '--> Pleurisy')
file = './out_new/all-all-223-2.0-0.99/name_space.csv'
if os.path.exists(file):
    go_idx = pd.read_csv(file, usecols=[1, 2, 3, 4, 5, 6])
go_idx

pleural pain -- No.223, the only side effect which is not associated any biological process GO item.

pleural pain --> Pleurisy


,name,namespace,id,p_fdr_bh,gene,symbol
0,voltage-gated calcium channel complex,cellular_component,GO:0005891,0.000942,57685,CACHD1
1,voltage-gated calcium channel complex,cellular_component,GO:0005891,0.000942,93589,CACNA2D4
2,voltage-gated calcium channel complex,cellular_component,GO:0005891,0.000942,55799,CACNA2D3
3,voltage-gated calcium channel activity,molecular_function,GO:0005245,0.007618,57685,CACHD1
4,voltage-gated calcium channel activity,molecular_function,GO:0005245,0.007618,93589,CACNA2D4
5,voltage-gated calcium channel activity,molecular_function,GO:0005245,0.007618,55799,CACNA2D3


In [107]:

data.dd_edge_index[0][data.dd_edge_index[1] == 12]

IndexError: The shape of the mask [2, 12062] at index 1 does not match the shape of the indexed tensor [2, 387] at index 1

In [114]:
for i in range(861):
    tmp = data.dd_edge_index[0]
    if 88 in tmp[1][np.where(tmp[0] == 12)]:
        print(f"i: {i} -- {se_idx2name[i]}")

#### C1 General M

### Case Study II: Mental Discorders

In [15]:
md_ids = [se_name2idx[i] for i in psychiaty_psychology]
list(zip(psychiaty_psychology, md_ids))

[('panic disorder', 452),
 ('anxiety', 330),
 ('bipolar disorder', 480),
 ('major depression', 597),
 ('alcohol consumption', 646),
 ('schizoaffective disorder', 664),
 ('borderline personality disorder', 665),
 ('agoraphobia', 703),
 ('conversion disorder', 792)]

In [ ]:
#### Panic Disorder

panicdisorder = pd.read_csv('c1_out/all-all-452')